In [56]:
import mne
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib qt

In [57]:
raw = mne.io.read_raw_edf('/Users/fan/Documents/Fanqingchen/yy/yy2.edf')
raw.set_channel_types({'EOG G_01-REF':'eog','EOG G_02-REF':'eog','EMG G_03-REF':'emg'})

Extracting EDF parameters from /Users/fan/Documents/Fanqingchen/yy/yy1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<RawEDF | yy1.edf, 37 x 86016 (168.0 s), ~40 kB, data not loaded>

In [58]:
raw.plot(start=2,duration=2)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [ ]:
#分割事件
#分割前要copy一份数据，因为剪切不可逆
raw_cropped = raw.copy()
raw_cropped.info

In [59]:
#定位电极
raw_cropped.set_montage('standard_1020',on_missing='warn')

/var/folders/hb/6pn1n6c164vbs0z_6nwtr7mw0000gn/T/ipykernel_25545/3847922987.py:2: RuntimeWarning: DigMontage is only a subset of info. There are 34 channel positions not present in the DigMontage. The required channels are:

['EEG Fp1-REF', 'EEG Fp2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG Fz-REF', 'EEG Cz-REF', 'EEG Pz-REF', 'EEG FC1-REF', 'EEG FC2-REF', 'EEG CP1-REF', 'EEG CP2-REF', 'EEG PO3-REF', 'EEG PO4-REF', 'EEG FC5-REF', 'EEG FC6-REF', 'EEG CP5-REF', 'EEG CP6-REF', 'EEG PO7-REF', 'EEG PO8-REF', 'EEG Oz-REF'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw_cropped.set_montage('standard_1020',on_missing='warn')


<RawEDF | yy1.edf, 37 x 86016 (168.0 s), ~41 kB, data not loaded>

In [60]:
#插值坏导
##raw_cropped.load_data()
##raw_cropped.interpolate_bads()
#展示插值坏导后的结果
##raw_cropped.plot()
##plt.show()

In [61]:
#重参考
raw_ref = raw_cropped.copy()
raw_ref.load_data()
raw_ref.set_eeg_reference(ref_channels=['EEG F3-REF','EEG F4-REF']) #选择哪一个位置作为重参考标准！


Reading 0 ... 86015  =      0.000 ...   167.998 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<RawEDF | yy1.edf, 37 x 86016 (168.0 s), ~24.3 MB, data loaded>

In [62]:
#滤波（不可逆操作，先copy）,这里要做高通、还是低通，会有不同选择
raw_filter = raw_ref.copy()
raw_filter.filter(l_freq=1,h_freq=None) #低通滤波
raw_filter.plot_psd(fmax=60)#功率谱密度图
plt.show()

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    0.1s finished


Effective window size : 4.000 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
/var/folders/hb/6pn1n6c164vbs0z_6nwtr7mw0000gn/T/ipykernel_25545/3092658829.py:4: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw_filter.plot_psd(fmax=60)#功率谱密度图


In [ ]:
#去伪迹ICA
ica = mne.preprocessing.ICA(n_components=34,method='fastica',max_iter=800)
ica.fit(raw_filter)
raw_filter.load_data()
ica.plot_sources(raw_filter,show_scrollbars=True,block=True)#展示出成分，可以手动去掉伪迹（坏导）
plt.show()



Fitting ICA to data using 34 channels (please be patient, this may take a while)
Selecting by number: 34 components
Fitting ICA took 5.1s.
Creating RawArray with float64 data, n_channels=36, n_times=86016
    Range : 0 ... 86015 =      0.000 ...   167.998 secs
Ready.


/var/folders/hb/6pn1n6c164vbs0z_6nwtr7mw0000gn/T/ipykernel_25545/4080550256.py:3: RuntimeWarning: Using n_components=34 (resulting in n_components_=34) may lead to an unstable mixing matrix estimation because the ratio between the largest (33) and smallest (6.5e-30) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 32
  ica.fit(raw_filter)


Channels marked as bad:
none


In [ ]:
print(ica)


In [ ]:
raw_recons = raw_filter.copy()
raw_recons = ica.apply(raw_recons)
raw_filter.plot()#ica前
raw_recons.plot()#ica后
plt.show()

In [ ]:
#高通滤波与凹陷滤波(这里的参数具体设置多少，按实际数据来设置)
raw_recons.filter(l_freq=None,h_freq=30)
raw_recons.notch_filter(freqs=50)


In [ ]:
#提取epoch
events = mne.events_from_annotations(raw_recons)
events_dic = {'pos':events[1]['16']}
#reject_criteria = dict(eeg=100e-6)#100μv 去除振幅大于100μv的
epochs = mne.Epochs(raw_recons,events[0],event_id=events_dic,preload=True,tmax=1,tmin-0.2)
epochs.plot(events=events[0])#目视检查
plt.show()

In [ ]:
pos_epochs = epochs['pos'] #'pos'人为定义名字，标记的annotation
neg_epochs = epochs['neg']#'pos'人为定义名字，标记的annotation

pos_evoked = pos_epochs.average()
neg_evoked = neg_epochs.average()
neg_evoked.plot()
